In [1]:
##Step4

import pandas as pd
from openpyxl import load_workbook
# 加载数据
file_path = "E:/My_python_work_xibanya/LG4processed.xlsx"
sheet_name = "Parent1"
df = pd.read_excel(file_path, sheet_name=sheet_name)

# 根据第一列的数字正向排序
df_sorted = df.sort_values(by=df.columns[0], ascending=True)

import pandas as pd

# 指定文件路径
file_path = "E:/My_python_work_xibanya/LG4processed.xlsx"

# 加载 'Parent1' 工作表
df = pd.read_excel(file_path, sheet_name='Parent1')

# 进行你的数据处理...
# 例如，根据第一列正向排序
df_sorted = df.sort_values(by=df.columns[0])

# 计算第二列中每个染色体标记出现的频次
chromosome_counts = df_sorted.iloc[:, 1].value_counts()
# 找到出现最多的染色体标记
most_common_chromosome = chromosome_counts.idxmax()

# 筛选不是出现频次最高染色体标记的行
df_not_most_common = df_sorted[df_sorted.iloc[:, 1] != most_common_chromosome]

# 将剩余的行（即是出现频次最高染色体标记的行）保留在原DataFrame中
df_sorted = df_sorted[df_sorted.iloc[:, 1] == most_common_chromosome]



# 分析第六列前100行中'A'和'H'的出现频率
letter_counts = df_sorted.iloc[:100, 5].value_counts()  # 假设第六列的索引为5

# 默认排序为A的占多数的情况
sort_ascending = True  # A占多数时为True，H占多数时为False

# 判断哪个字母占大多数，并设置相应的排序策略
if letter_counts.get('H', 0) > letter_counts.get('A', 0):
    sort_ascending = False
# 分析第六列前100行中'A'和'H'的出现频率
letter_counts = df_sorted.iloc[:100, 5].value_counts()  # 假设第六列的索引为5

# 默认排序为A的占多数的情况
sort_ascending = True  # A占多数时为True，H占多数时为False

# 判断哪个字母占大多数，并设置相应的排序策略
if letter_counts.get('H', 0) > letter_counts.get('A', 0):
    sort_ascending = False
    
# 执行排序
# 若A占大多数，则第六列正序排序，第一列也正序排序
# 若H占大多数，则第六列逆序排序，第一列正序排序
df_sorted = df_sorted.sort_values(by=[df_sorted.columns[5], df_sorted.columns[0]], 
                                  ascending=[sort_ascending, True])
# 步骤 1: 找到值不再递增的起始行
df=df_sorted
start_row = 0
for i in range(1, len(df)):
    if df.iloc[i, 0] < df.iloc[i-1, 0]:
        start_row = i
        break

# 如果没有发现递增顺序被打断的情况，start_row将保持为0，意味着不需要进行替换操作
if start_row > 0:
    # 步骤 2: 进行字母替换
    for i in range(start_row, len(df)):
        for j in range(5, df.shape[1]):  # 从第六列开始
            cell_value = df.iloc[i, j]
            if cell_value == 'A':
                df.iloc[i, j] = 'K'  # 首先，将所有的A替换成K
            elif cell_value == 'H':
                df.iloc[i, j] = 'A'  # 然后，将所有的H替换成A

    # 现在将所有的K替换成H
    for i in range(start_row, len(df)):
        for j in range(5, df.shape[1]):  # 从第六列开始
            if df.iloc[i, j] == 'K':
                df.iloc[i, j] = 'H'

#  重新排序
df = df.sort_values(by=df.columns[0])

# . 删除旧的计数列和函数列（如果存在）
for col in ['A_Count', 'B_Count', 'H_Count', '-_Count', 'f(A)', 'f(H)']:
    if col in df.columns:
        df.drop(col, axis=1, inplace=True)

# . 重新计算各计数列
# 定义计数函数
def count_letters_from_sixth_column(row, letter):
    return (row.iloc[5:] == letter).sum()  # 从第六列开始计数

df['A_Count'] = df.apply(lambda row: count_letters_from_sixth_column(row, 'A'), axis=1)
df['B_Count'] = df.apply(lambda row: count_letters_from_sixth_column(row, 'B'), axis=1)
df['H_Count'] = df.apply(lambda row: count_letters_from_sixth_column(row, 'H'), axis=1)
df['-_Count'] = df.apply(lambda row: count_letters_from_sixth_column(row, '-'), axis=1)

# . 重新计算 f(A) 和 f(H)
df['f(A)'] = df.apply(lambda row: (2 * row['A_Count'] + row['H_Count']) / (2 * (row['A_Count'] + row['B_Count'] + row['H_Count'])) if (row['A_Count'] + row['B_Count'] + row['H_Count']) > 0 else np.nan, axis=1)
df['f(H)'] = df.apply(lambda row: row['H_Count'] / (row['A_Count'] + row['B_Count'] + row['H_Count']) if (row['A_Count'] + row['B_Count'] + row['H_Count']) > 0 else np.nan, axis=1)


file_path = "E:/My_python_work_xibanya/LG4processed.xlsx"
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df.to_excel(writer, sheet_name="Parent1-1", index=False)
    df_not_most_common.to_excel(writer, sheet_name="DSC.1", index=False)
file_path = "E:/My_python_work_xibanya/LG4processed.xlsx"
sheet_name = "Parent1-1"
df = pd.read_excel(file_path, sheet_name=sheet_name)
import pandas as pd

import numpy as np

# 假设df是已经加载的DataFrame，并且每个单元格只包含字母A或H

def compute_difference(row1, row2):
    """计算两行之间的差异度"""
    return sum(a != b for a, b in zip(row1, row2))

def flip_letters(s):
    """转换行中的A和H"""
    return s.replace({'A': 'H', 'H': 'A'})

# 初始化一个列表来保存优化后的行
optimized_rows = [df.iloc[0]]

for i in range(1, len(df)):
    current_row = df.iloc[i]
    prev_row = optimized_rows[-1]
    
    # 计算当前行与上一行的差异
    diff_with_current = compute_difference(prev_row[5:], current_row[5:])
    
    # 计算如果转换当前行后与上一行的差异
    flipped_current_row = flip_letters(current_row[5:])
    diff_with_flipped = compute_difference(prev_row[5:], flipped_current_row)
    
    # 如果转换后差异更小，则保存转换后的行
    if diff_with_flipped < diff_with_current:
        optimized_rows.append(current_row.apply(lambda x: 'H' if x == 'A' else 'A' if x == 'H' else x))
    else:
        optimized_rows.append(current_row)

# 将优化后的行组合成新的DataFrame
optimized_df = pd.DataFrame(optimized_rows).reset_index(drop=True)

file_path = "E:/My_python_work_xibanya/LG4processed.xlsx"
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    optimized_df .to_excel(writer, sheet_name="Parent1-2", index=False)
    
file_path = "E:/My_python_work_xibanya/LG4processed.xlsx"
sheet_name = "Parent1-2"
df = pd.read_excel(file_path, sheet_name=sheet_name)

import pandas as pd
import numpy as np

# 假设df是您的DataFrame，并且每个单元格只包含'A'或'H'

def similarity(row1, row2):
    """计算两行间的相似度"""
    return sum(a == b for a, b in zip(row1, row2))

def flip_row(row):
    """在行中交换A和H"""
    return ['H' if cell == 'A' else 'A' if cell == 'H' else cell for cell in row]

# 初始化结果DataFrame，首行不变
optimized_rows = [df.iloc[0, 5:].tolist()]

# 遍历DataFrame的行，比较相邻行的相似度
for i in range(1, len(df)):
    current_row = df.iloc[i, 5:].tolist()
    prev_optimized_row = optimized_rows[-1]
    
    # 计算不交换的相似度
    sim_without_flip = similarity(prev_optimized_row, current_row)
    # 计算交换后的相似度
    flipped_row = flip_row(current_row)
    sim_with_flip = similarity(prev_optimized_row, flipped_row)
    
    # 根据相似度是否提升来决定是否交换
    if sim_with_flip > sim_without_flip:
        optimized_rows.append(flipped_row)
    else:
        optimized_rows.append(current_row)

# 用优化后的行数据更新DataFrame（只更新第六列及以后的列）
for i, row in enumerate(optimized_rows, start=0):
    df.iloc[i, 5:] = row
with pd.ExcelWriter("E:/My_python_work_xibanya/LG4processed.xlsx", engine='openpyxl', mode='a') as writer:
    df.to_excel(writer, sheet_name='Parent1-3', index=False)
   